# IS434 Twitter/X Bot Detection Project

This Project aims to use machine learning and analytics techniques to detect bot accounts on the Twitter/X platform 

# 0. Install Libraries

In [1]:
!pip install pgmpy
!pip install gensim
!pip install nltk
!pip install imblearn
!pip install keras
!pip install scikit-learn
!pip install tensorflow

  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl (431 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl.metadata (541 bytes)
Using cached protobuf-4.25.5-cp310-abi3-win_amd64.whl (413 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.66.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.5 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Data Processing

In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Step 1: Merge the CSV Files
# Load the CSV files (adjust file names and paths as necessary)
tweets_df = pd.read_csv('data/tweet_labelled_cleaned_topics.csv')  # Contains 'tweet_text' and 'topic_number'
tags_df = pd.read_csv('data/labels.csv')      

# Merge on 'topic_number'
merged_df = pd.merge(tweets_df, tags_df, on='tweet_topics', how='left')

# Step 2: Prepare the Tags for One-Hot Encoding
merged_df['tags'] = merged_df['tags'].apply(lambda x: x.lower().split(','))

# Step 3: One-Hot Encode the Tags
mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(merged_df['tags'])
tags_encoded_df = pd.DataFrame(tags_encoded, columns=mlb.classes_)
merged_df = pd.concat([merged_df, tags_encoded_df], axis=1)
merged_df.drop(columns=['tags','index','tweet_topics','label'], inplace=True)

save_path = 'data/merged_df.csv'
merged_df.to_csv(save_path, index=False)

merged_df.head()

,tweet,processed_tweet,ableism,anti_religion,harm,homophobia,islamophobia,lookism,political_polarisation,racism,religious_intolerance,sexism,vulgarity,xenophobia
0,` :Click on my ``Annoying Users`` link! I gue...,"['click', 'annoying', 'user', 'link', 'guess',...",0,0,0,0,0,0,1,0,0,0,1,0
1,Iraq is not good === === USA is bad,"['iraq', 'good', 'usa', 'bad']",0,0,0,0,0,0,1,0,0,0,1,0
2,` Buddha - ``Some suggest that victims should...,"['buddha', 'suggest', 'victim', 'referred', 'a...",0,0,0,0,0,0,1,0,0,0,1,0
3,____ fuck off you little asshole. If you wan...,"['____', 'fuck', 'little', 'asshole', 'want', ...",1,0,0,1,0,0,0,1,0,0,1,1
4,"i have a dick, its bigger than yours! hahaha","['dick', 'bigger', 'hahaha']",1,0,0,1,0,0,0,1,0,0,1,1


Split test and val set

In [24]:
import pandas as pd
merged_df = pd.read_csv('data/merged_df.csv')

# Modelling 

In [25]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import precision_recall_fscore_support, classification_report
import numpy as np
from transformers import BertTokenizer, BertModel
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Label columns (using your existing ones)
label_columns = [
    "ableism", "anti_religion", "harm", "homophobia", "islamophobia", "lookism",
    "political_polarisation", "racism", "religious_intolerance", "sexism", "vulgarity", "xenophobia"
]

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Clean text
        text = self._clean_text(text)
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.float)
        }
    
    def _clean_text(self, text):
        # Basic text cleaning
        text = text.lower().strip()
        # Remove URLs
        text = ' '.join(word for word in text.split() 
                       if not word.startswith('http') and not word.startswith('www'))
        return text

class LSTMHateSpeech(nn.Module):
    def __init__(self, num_labels):
        super(LSTMHateSpeech, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        
        # Multiple dense layers with batch normalization
        self.dense1 = nn.Linear(768, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dense2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.classifier = nn.Linear(256, num_labels)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        
        x = torch.relu(self.bn1(self.dense1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.dense2(x)))
        x = self.dropout(x)
        
        return self.classifier(x)

# Prepare data
texts = merged_df['tweet'].values
labels = merged_df[label_columns].values

# Calculate dataset sizes
dataset_size = len(texts)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create full dataset
full_dataset = TweetDataset(texts, labels, tokenizer)

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Calculate class weights for handling imbalance
def calculate_class_weights(labels):
    pos_weights = []
    for i in range(labels.shape[1]):
        neg_count = len(labels[labels[:, i] == 0])
        pos_count = len(labels[labels[:, i] == 1])
        pos_weights.append((neg_count + 1) / (pos_count + 1))
    return torch.tensor(pos_weights, dtype=torch.float).to(device)

class_weights = calculate_class_weights(labels)

# Initialize model and training components
model = LSTMHateSpeech(len(label_columns)).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

# Training and evaluation functions
def evaluate_model(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask, token_type_ids)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.sigmoid(outputs) >= 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    metrics = classification_report(
        np.array(all_labels), 
        np.array(all_preds), 
        target_names=label_columns,
        output_dict=True
    )
    
    return total_loss / len(data_loader), metrics

# Training loop
num_epochs = 10
best_val_f1 = 0
best_model = None
patience = 3
patience_counter = 0

for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # Validation phase
    val_loss, val_metrics = evaluate_model(model, val_loader, criterion)
    val_f1 = val_metrics['macro avg']['f1-score']
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Average Training Loss: {avg_train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Macro F1: {val_f1:.4f}")
    
    # Early stopping with model saving
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        best_model = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

# Load best model
model.load_state_dict(best_model)

# Final evaluation on test set
test_loss, test_metrics = evaluate_model(model, test_loader, criterion)
print("\nTest Set Performance:")
print(f"Test Loss: {test_loss:.4f}")
print("\nDetailed Metrics:")
for label in label_columns:
    print(f"\n{label}:")
    print(f"Precision: {test_metrics[label]['precision']:.4f}")
    print(f"Recall: {test_metrics[label]['recall']:.4f}")
    print(f"F1-score: {test_metrics[label]['f1-score']:.4f}")

# Save the model
torch.save(model.state_dict(), 'hate_speech_model.pth')


Using device: cuda


c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

Epoch 1/10
Average Training Loss: 0.6173
Validation Loss: 0.4162
Validation Macro F1: 0.7291


c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

Epoch 2/10
Average Training Loss: 0.3715
Validation Loss: 0.3466
Validation Macro F1: 0.8071


...

Epoch 9/10
Average Training Loss: 0.0407
Validation Loss: 0.5654
Validation Macro F1: 0.8995


c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

Epoch 10/10
Average Training Loss: 0.0390
Validation Loss: 0.5873
Validation Macro F1: 0.9002


c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\benhz\Documents\GitHub\IS434\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is


Test Set Performance:
Test Loss: 0.5583

Detailed Metrics:

ableism:
Precision: 0.8984
Recall: 0.9074
F1-score: 0.9029

anti_religion:
Precision: 0.8489
Recall: 0.8741
F1-score: 0.8613

harm:
Precision: 0.9027
Recall: 0.8855
F1-score: 0.8940

homophobia:
Precision: 0.8835
Recall: 0.8955
F1-score: 0.8895

islamophobia:
Precision: 0.8225
Recall: 0.8203
F1-score: 0.8214

lookism:
Precision: 0.9027
Recall: 0.8855
F1-score: 0.8940

political_polarisation:
Precision: 0.9109
Recall: 0.9224
F1-score: 0.9166

racism:
Precision: 0.9246
Recall: 0.9202
F1-score: 0.9224

religious_intolerance:
Precision: 0.8956
Recall: 0.9047
F1-score: 0.9001

sexism:
Precision: 0.9161
Recall: 0.9024
F1-score: 0.9092

vulgarity:
Precision: 0.9499
Recall: 0.9583
F1-score: 0.9541

xenophobia:
Precision: 0.9052
Recall: 0.9063
F1-score: 0.9058


In [26]:
torch.save(model.state_dict(), "model/hate_speech_model.pth")

Model saved as bilstm_attention_model.pth


### Predictions

In [27]:
# Load the model
model = LSTMHateSpeech(len(label_columns)).to(device)
model.load_state_dict(torch.load('hate_speech_model.pth'))
model.eval()

C:\Users\benhz\AppData\Local\Temp\ipykernel_27624\3778486265.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('hate_speech_model.pth'))


ImprovedHateSpeechModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [36]:
def predict(tweet, model, tokenizer, threshold=0.7):
    model.eval()
    encoded = tokenizer.encode_plus(
        tweet,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors="pt"
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    token_type_ids = encoded['token_type_ids'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask, token_type_ids)
    
    preds = torch.sigmoid(outputs) >= threshold
    pred_labels = [label_columns[i] for i, pred in enumerate(preds.squeeze()) if pred == 1]

    return pred_labels

In [37]:
tweets = [
    "All Chinese people are disgusting",
    "Exams are coming, and I am not ready",
    "Children are so annoying",
    "I hate my job",
    "Chinese should go back to their country",
    "Asians should not be allowed to drive",
    "The storm last night was crazy"
]

for tweet in tweets:
    print(f"Tweet: {tweet}")
    predictions = predict(tweet, model, tokenizer)
    print(tweet)
    print(f"Predicted Labels: {predictions}\n")

Tweet: All Chinese people are disgusting
All Chinese people are disgusting
Predicted Labels: ['ableism', 'homophobia', 'racism', 'vulgarity', 'xenophobia']

Tweet: Exams are coming, and I am not ready
Exams are coming, and I am not ready
Predicted Labels: []

Tweet: Children are so annoying
Children are so annoying
Predicted Labels: ['political_polarisation', 'vulgarity']

Tweet: I hate my job
I hate my job
Predicted Labels: ['islamophobia', 'political_polarisation', 'racism', 'xenophobia']

Tweet: Chinese should go back to their country
Chinese should go back to their country
Predicted Labels: ['ableism', 'homophobia', 'racism', 'vulgarity', 'xenophobia']

Tweet: Asians should not be allowed to drive
Asians should not be allowed to drive
Predicted Labels: ['political_polarisation', 'vulgarity']

Tweet: The storm last night was crazy
The storm last night was crazy
Predicted Labels: []

